In [2]:
# On affiche les graphiques dans le notebook en statique
%matplotlib inline

In [3]:
import numpy as np
import logging
import re
import os
import gc
import joblib
import lightgbm as lgb
import optuna
import plotly
import kaleido
import mlflow

import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, make_scorer, f1_score

In [4]:
from src.p7_constantes import DATA_INTERIM, DATA_BASE, MODEL_DIR
from src.p7_constantes import LOCAL_HOST, LOCAL_PORT
from src.p7_util import timer, clean_ram
from src.p7_regex import sel_var

In [5]:
print("mlflow", mlflow.__version__)
print("optuna", optuna.__version__)
print("numpy", np.__version__)
print("plotly", plotly.__version__)
print("kaleido", kaleido.__version__)

mlflow 2.12.1
optuna 3.6.1
numpy 1.26.4
plotly 5.21.0
kaleido 0.1.0


Démarrer

In [6]:
# from src.p7_regex import sel_var

In [7]:
subdir = "light_simple/"

In [8]:
df = pd.read_csv(os.path.join(DATA_INTERIM, "all_data_simple_kernel_ohe.csv"))

to_drop = sel_var(df.columns, 'Unnamed')
if to_drop:
    df = df.drop(to_drop, axis=1)

df = df.rename(columns=lambda x: re.sub("[^A-Za-z0-9_]+", "", x))
    
train = df[df['TARGET'].notnull()]
#test = df[df['TARGET'].isnull()]

del df
gc.collect()

print("Forme de train.csv :", train.shape)
#print("Forme de test :", test.shape)
train.head()

0 variables à inclure correspondant au motif 'Unnamed' : []
0 variables à exclure correspondant au motif 'None' : []
0 variables sélectionnées : []
Forme de train.csv : (307507, 794)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Sentproposal_MEAN,CC_NAME_CONTRACT_STATUS_Sentproposal_SUM,CC_NAME_CONTRACT_STATUS_Sentproposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#from src.p7_util import clean_ram

In [10]:
dic_local = locals()
to_del = ['to_drop', 'selected_features']
clean_ram(to_del, dic_local)

1 variables détruites : ['to_drop']


In [11]:
# Passe les variables catégorielles en dtype category sinon LGM ne pourra pas les traiter
cat_features = list(train.loc[:, train.dtypes == 'object'].columns.values)
for feature in cat_features:
    train[feature] = pd.Series(train[feature], dtype="category")

Récupération des features par ordre d'importance

In [12]:
sorted_features_by_importance = pd.read_csv(os.path.join(MODEL_DIR, subdir, "feature_importance.csv")).set_index("feature").index.tolist()
print(len(sorted_features_by_importance))
sorted_features_by_importance[:10]

792


['PAYMENT_RATE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_3',
 'EXT_SOURCE_2',
 'DAYS_BIRTH',
 'AMT_ANNUITY',
 'DAYS_EMPLOYED',
 'APPROVED_CNT_PAYMENT_MEAN',
 'DAYS_ID_PUBLISH',
 'ACTIVE_DAYS_CREDIT_MAX']

In [11]:
"""# Essai sur un échantillon d'abord
n_rows = None
X = train.drop(columns=["SK_ID_CURR", "TARGET"], axis=1)
y = train["TARGET"]
if n_rows:
    X = X.head(n_rows)
    y = y.head(n_rows)"""

'# Essai sur un échantillon d\'abord\nn_rows = None\nX = train.drop(columns=["SK_ID_CURR", "TARGET"], axis=1)\ny = train["TARGET"]\nif n_rows:\n    X = X.head(n_rows)\n    y = y.head(n_rows)'

In [12]:
"""predictors = list(X.columns)
len(predictors)"""

'predictors = list(X.columns)\nlen(predictors)'

In [14]:
not_predictors = [
    'TARGET',
    "SK_ID_CURR",
    "SK_ID_BUREAU",
    "SK_ID_PREV",
    "index",
    "level_0",
    ]
predictors = list(filter(lambda v: v not in not_predictors, train.columns))

In [15]:
len(predictors)

792

In [15]:
"""X.info()"""

'X.info()'

In [16]:
n_features = 20

In [17]:
predictors = predictors[:n_features]
len(predictors)

20

# Métrique personnalisée

In [17]:

# Pénalise les Faux Negatifs dans le F1 Score
def weight_f1(y_true, y_pred, weight_fn=10):
    _, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    # fn = Nombre de faux négatifs (oubli de prédire un défaut)
    # fp = Nombre de faux positifs (défaut prédit à tort)
    
    # f1 standard = 2 * tp / (2 * tp + fp + fn)
    
    #weighted_f1 = 2 * tp / (2 * tp + weight_fn * fn  + (1 - weight_fn) * fp)
    weighted_f1 = 2 * tp / (2 * tp + (weight_fn * fn  +  fp) / weight_fn)
    
    return weighted_f1

# Créer une métrique personnalisée à partir de la fonction de perte
custom_f1 = make_scorer(weight_f1, greater_is_better=True)

## Run A Study

In [18]:
"""
La fonction _objectif est appelée une fois pour chaque essai (trial).
Ici on entraîne un LGBMClassfier et on calcule la métrique : ? replacer le f1-score par une autre
Optuna passe un objet trial à la fonction _objectif, qu'on peut utiliser pour en définir les paramètres.
log=True : applique une log scale aux valeurs à tester dans l'étendue spécifiée (pour les valeurs num), 
Effet : plus de valeurs sont testées à proximité de la borne basse et moins (logarithmiquement) vers
la borne haute
Convient particulièrement bien au learning rate : on veut se concentrer sur des valeurs + petites et 
augmenter exponentiellement le pas des valeurs à tester pour les plus grandes

Le pruning callbac est le mécanisme qui applique le pruning dynamique pendant l'entraînement du modèle LightGBM 
dans le cadre d'une étude Optuna. 
1 - Initialisation du callback : 
Crée le callback de pruning qui surveillera l'entraînement du modèle LightGBM 
et effectuera le pruning selon les instructions spécifiées par l'étude Optuna
2 - Évaluation périodique : 
Pendant l'entraînement de LightGBM, le callback est appelé périodiquement
pour évaluer les performances du modèle en fonction des critères de pruning définis par Optuna.
3 - Décision de pruning : 
Le callback utilise les informations fournies par Optuna, 
telles que les valeurs d'objectif de l'essai actuel et les valeurs d'objectif des essais précédents, 
pour décider si l'essai actuel doit être pruned en fonction de son efficacité par rapport à d'autres essais.
4 - Pruning : 
Pruning appliqué si décidé (arrête l'entraînement de ce modèle LightGBM)
5 - Réévaluation et ajustement :
Après le pruning d'un essai, l'entraînement peut continuer avec les essais restants, 
et le processus de pruning peut être répété à intervalles réguliers jusqu'à ce que l'étude soit terminée.


"""

"\nLa fonction _objectif est appelée une fois pour chaque essai (trial).\nIci on entraîne un LGBMClassfier et on calcule la métrique : ? replacer le f1-score par une autre\nOptuna passe un objet trial à la fonction _objectif, qu'on peut utiliser pour en définir les paramètres.\nlog=True : applique une log scale aux valeurs à tester dans l'étendue spécifiée (pour les valeurs num), \nEffet : plus de valeurs sont testées à proximité de la borne basse et moins (logarithmiquement) vers\nla borne haute\nConvient particulièrement bien au learning rate : on veut se concentrer sur des valeurs + petites et \naugmenter exponentiellement le pas des valeurs à tester pour les plus grandes\n\nLe pruning callbac est le mécanisme qui applique le pruning dynamique pendant l'entraînement du modèle LightGBM \ndans le cadre d'une étude Optuna. \n1 - Initialisation du callback : \nCrée le callback de pruning qui surveillera l'entraînement du modèle LightGBM \net effectuera le pruning selon les instructions 

In [19]:
#from sklearn.model_selection import cross_validate

In [20]:
#from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import f1_score

In [76]:
from sklearn.metrics import recall_score, precision_recall_fscore_support

In [88]:
# Nécessite l'installation optuna-integration
def objective(optimize_boosting_type=True):
    def _objective(trial):
        with mlflow.start_run(run_name=f"T_{trial._trial_id}", nested=True):
            if optimize_boosting_type:
                boosting_type = trial.suggest_categorical("boosting_type", ["dart", "gbdt"])
            else:
                boosting_type = "gbdt"
            lambda_l1 = trial.suggest_float(
                'lambda_l1', 1e-8, 10.0, log=True),
            lambda_l2 = trial.suggest_float(
                'lambda_l2', 1e-8, 10.0, log=True),
            num_leaves = trial.suggest_int(
                'num_leaves', 2, 256),
            feature_fraction = trial.suggest_float(
                'feature_fraction', 0.4, 1.0),
            bagging_fraction = trial.suggest_float(
                'bagging_fraction', 0.4, 1.0),
            bagging_freq = trial.suggest_int(
                'bagging_freq', 1, 7),
            min_child_samples = trial.suggest_int(
                'min_child_samples', 5, 100),
            learning_rate = trial.suggest_float(
                "learning_rate", 0.0001, 0.5, log=True),
            max_bin = trial.suggest_int(
                "max_bin", 128, 512, step=32)
            n_estimators = trial.suggest_int(
                "n_estimators", 40, 400, step=20)
            
            hyperparams = {
                'optimize_boosting_type': optimize_boosting_type,
                'lambda_l1': lambda_l1,
                'lambda_l2': lambda_l2,
                'num_leaves': num_leaves,
                'feature_fraction': feature_fraction,
                'bagging_fraction': bagging_fraction,
                'bagging_freq': bagging_freq,
                'min_child_samples': min_child_samples,
                'learning_rate': learning_rate,
                'max_bin': max_bin,
                'n_estimators': n_estimators,
            }
            
            # 'binary' est la métrique d'erreur
            pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "binary")
            
            
            
            
            # Pour intégration optuna mlflow avec nested runs voir :
            # https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/hyperparameter-tuning-with-child-runs.html?highlight=run%20description

            # On n'utilise pas cross_val_score pour des problèmes de RAM
            #scores = cross_val_score(model, X, y, scoring="f1_macro", cv=5)
            folds = StratifiedKFold(
                n_splits=5, shuffle=True, random_state=42
            )
            f1_scores = []
            weighted_recall_scores = []
            
            for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[predictors], train["TARGET"])):
                train_x, train_y = train[predictors].iloc[train_idx], train["TARGET"].iloc[train_idx]
                valid_x, valid_y = train[predictors].iloc[valid_idx], train["TARGET"].iloc[valid_idx]
                
                model = lgb.LGBMClassifier(
                    force_row_wise=True,
                    boosting_type=boosting_type,
                    n_estimators=n_estimators,
                    lambda_l1=lambda_l1,
                    lambda_l2=lambda_l2,
                    num_leaves=num_leaves,
                    feature_fraction=feature_fraction,
                    bagging_fraction=bagging_fraction,
                    bagging_freq=bagging_freq,
                    min_child_samples=min_child_samples,
                    learning_rate=learning_rate,
                    max_bin=max_bin,
                    callbacks=[pruning_callback],
                    verbose=-1,
                )
                
                model.fit(
                    train_x,
                    train_y,
                    eval_set=[(train_x, train_y), (valid_x, valid_y)],
                    eval_metric="f1_macro",
                )
                
                pred_y = model.predict(valid_x)
                f1_scores.append(f1_score(valid_y, pred_y))
                weighted_recall_scores.append(recall_score(valid_y, pred_y, average='weighted'))
            mean_f1_scores = np.mean(f1_scores)
            mean_weighted_recall_scores = np.mean(weighted_recall_scores)
            
            """dic_metrics = {
                "f1_score": mean_f1_scores,
                "weighted_recall_score": mean_weighted_recall_scores,
            }
            """
            mlflow.log_metric("f1_score", mean_f1_scores)
            mlflow.log_metric("weighted_recall", mean_weighted_recall_scores)
            mlflow.log_params(hyperparams)
        return mean_weighted_recall_scores

    return _objective

In [18]:
"""
Métriques pour notre cas :

Le Recall mesure la proportion de vrais positifs (défauts correctement prédits)
parmi tous les vrais positifs et faux négatifs (défauts réels).
En choisissant le Recall comme métrique, on se concentre sur la capacité du modèle à capturer la majorité des cas de défauts, 
minimisant ainsi le risque de faux négatifs (ne pas prédire un défaut lorsqu'il existe réellement).

Le F1-score est une mesure combinée de la précision et du recall. 
Il s'agit de la moyenne harmonique de la précision et du recall.
Le F1-score est particulièrement utile lorsque les classes sont déséquilibrées car il donne plus de poids aux classes minoritaires. 
Ainsi, il peut être une bonne métrique lorsqu'on veut trouver un compromis entre la précision et le recall.

AUC-ROC (Area Under the Receiver Operating Characteristic Curve) : L'AUC-ROC est une métrique qui mesure la capacité du modèle à classer
correctement les exemples positifs et négatifs. 
Elle est robuste aux déséquilibres de classe et donne une indication de la capacité du modèle à discriminer entre les classes.
"""

"\nMétriques pour notre cas :\n\nLe Recall mesure la proportion de vrais positifs (défauts correctement prédits)\nparmi tous les vrais positifs et faux négatifs (défauts réels).\nEn choisissant le Recall comme métrique, on se concentre sur la capacité du modèle à capturer la majorité des cas de défauts, \nminimisant ainsi le risque de faux négatifs (ne pas prédire un défaut lorsqu'il existe réellement).\n\nLe F1-score est une mesure combinée de la précision et du recall. \nIl s'agit de la moyenne harmonique de la précision et du recall.\nLe F1-score est particulièrement utile lorsque les classes sont déséquilibrées car il donne plus de poids aux classes minoritaires. \nAinsi, il peut être une bonne métrique lorsqu'on veut trouver un compromis entre la précision et le recall.\n\nAUC-ROC (Area Under the Receiver Operating Characteristic Curve) : L'AUC-ROC est une métrique qui mesure la capacité du modèle à classer\ncorrectement les exemples positifs et négatifs. \nElle est robuste aux dé

In [19]:
"""
scoring avec cross_val_score :
The 'scoring' parameter of cross_val_score must be a str among pour la classification :
{
'accuracy', 'balanced_accuracy', 'f1', 
'jaccard_micro',
'jaccard_weighted',
'precision_micro',
'recall_samples', 
'precision_macro',
'jaccard', 
'recall_micro', 
'roc_auc_ovo', 
'f1_samples', 'recall_macro', 'precision_weighted', 'f1_weighted', 'roc_auc_ovr_weighted',
'f1_micro', 'roc_auc', 'f1_macro', 'precision_samples', 'recall', 'jaccard_samples', 'precision', 'average_precision', 'top_k_accuracy', 
'neg_log_loss', 'jaccard_macro', 'neg_brier_score', 'roc_auc_ovr', 'recall_weighted', 'roc_auc_ovo_weighted'
}, a callable or None.

"""

"\nscoring avec cross_val_score :\nThe 'scoring' parameter of cross_val_score must be a str among pour la classification :\n{\n'accuracy', 'balanced_accuracy', 'f1', \n'jaccard_micro',\n'jaccard_weighted',\n'precision_micro',\n'recall_samples', \n'precision_macro',\n'jaccard', \n'recall_micro', \n'roc_auc_ovo', \n'f1_samples', 'recall_macro', 'precision_weighted', 'f1_weighted', 'roc_auc_ovr_weighted',\n'f1_micro', 'roc_auc', 'f1_macro', 'precision_samples', 'recall', 'jaccard_samples', 'precision', 'average_precision', 'top_k_accuracy', \n'neg_log_loss', 'jaccard_macro', 'neg_brier_score', 'roc_auc_ovr', 'recall_weighted', 'roc_auc_ovo_weighted'\n}, a callable or None.\n\n"

In [20]:
"""
balanced accuracy
The balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. 
It is defined as the average of recall obtained on each class.
The best value is 1 and the worst value is 0 when adjusted=False.
"""

'\nbalanced accuracy\nThe balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. \nIt is defined as the average of recall obtained on each class.\nThe best value is 1 and the worst value is 0 when adjusted=False.\n'

In [17]:
cat_features

[]

In [22]:
"""
Pruning (="Elagage") = technique pour arrêter prématurément l'exécution (train + eval) si 
il est peu probable que l'essai conduise à une amélioreration significative des performances
(l'entraînement est stoppé si l'algo sous-performe)

**** HYPERBAND

Hyperband est une technique de pruning qui combine le pruning de type "Successive halving" et
d'autres techniques (random search, multi-bracket resource allocation strategy).

1 - Successive Halving (Demi-seuccessifs):
Au début de l'optimisation, Hyperband crée un grand nombre de configurations d'hyperparamètres
et entraîne chaque modèle pendant un petit nombre d'itérations (eochs). 
Ensuite, il évalue les performances de ces modèles et conserve uniquement les meilleurs. 
Il répète ce processus plusieurs fois, en doublant à chaque fois le nombre d'itérations, 
mais en réduisant de moitié le nombre de configurations conservées.

Ainsi les modèles prometteurs bénéficient de plus d'itérations pour converger vers de bonnes performances,
tandis que les modèles moins prometteurs sont élagués à chaque étape.

2 - Pruning basé sur la performance:
À chaque étape du Successive Halving, Hyperband évalue périodiquement les performances des modèles en formation
et décide de "pruner" les modèles moins prometteurs. 
Les modèles qui n'atteignent pas un certain seuil de performance sont arrêtés prématurément.

*** MULTI_BRACKET RESSOURCE ALLOCATION strategy (Allocation de ressource à plusieurs 'brackets' ou niveaux / échelon)

divise les ressources disponibles (telles que le nombre d'itérations, d'évaluations de modèles, etc.)
en plusieurs "brackets" ou "niveaux" et alloue ces ressources de manière dynamique en fonction de la performance
des modèles à chaque niveau.

1 - Multi-brackets
L'algorithme divise le budget total de ressources (par exemple, le nombre maximal d'itérations ou le temps total de calcul) en plusieurs "brackets" ou "échelons".
Chaque bracket représente une étape distincte de l'algorithme où différents ensembles d'hyperparamètres sont évalués.
Chaque bracket est caractérisé par un budget de ressources spécifique, qui peut être soit le nombre d'itérations, soit le temps de calcul. 
Les brackets initiaux ont un budget élevé tandis que les brackets ultérieurs ont des budgets plus bas.

2 - Resource allocation
Alloue dynamiquement les ressources aux config d'hyperparams
Dans chaque bracket, Hyperband commence par évaluer un grand nombre de configurations d'hyperparamètres pour un nombre restreint d'itérations. 
Ensuite, il élimine les configurations sous-performantes et alloue davantage de ressources aux configurations les plus prometteuses.

Within each bracket, successive halving is applied to iteratively eliminate underperforming configurations and
allocate more resources to the remaining promising ones.
At the begining of each bracket, a new set of hyperparam configurations is sampled using random search.

=> reduce risk of missing good config
more efficient and effective hyperparam tuning.
"""

'\nPruning (="Elagage") = technique pour arrêter prématurément l\'exécution (train + eval) si \nil est peu probable que l\'essai conduise à une amélioreration significative des performances\n(l\'entraînement est stoppé si l\'algo sous-performe)\n\n**** HYPERBAND\n\nHyperband est une technique de pruning qui combine le pruning de type "Successive halving" et\nd\'autres techniques (random search, multi-bracket resource allocation strategy).\n\n1 - Successive Halving (Demi-seuccessifs):\nAu début de l\'optimisation, Hyperband crée un grand nombre de configurations d\'hyperparamètres\net entraîne chaque modèle pendant un petit nombre d\'itérations (eochs). \nEnsuite, il évalue les performances de ces modèles et conserve uniquement les meilleurs. \nIl répète ce processus plusieurs fois, en doublant à chaque fois le nombre d\'itérations, \nmais en réduisant de moitié le nombre de configurations conservées.\n\nAinsi les modèles prometteurs bénéficient de plus d\'itérations pour converger vers

In [23]:
"""
SAMPLER TPE :
Tree-structured Parzen Estimator
TPESampler utilise une approche bayésienne pour estimer les distributions de probabilité des valeurs d'hyperparamètres,
en se basant sur les performances des essais précédents, 
afin de guider efficacement la recherche vers des régions prometteuses de l'espace des hyperparamètres.

1 - Initialisation des distributions :
L'algorithme TPE commence par définir des distributions de probabilité pour chaque hyperparamètre à optimiser.
Ces distributions peuvent être continues (comme des distributions gaussiennes) ou discrètes (comme des distributions uniformes).
Pour chaque hyperparamètre, deux distributions sont définies : 
une pour les valeurs considérées comme "bonnes" (positive), et une pour les valeurs considérées comme "mauvaises" (négative).

2 - Évaluation des essais :
À chaque étape de l'optimisation, 
TPE utilise les essais précédents pour estimer les distributions de probabilité des valeurs d'hyperparamètres.
Les essais sont divisés en deux groupes : 
ceux qui ont produit de bonnes performances et ceux qui ont produit de mauvaises performances, 
en fonction du critère d'objectif (par exemple, la précision d'un modèle).
Les distributions de probabilité sont mises à jour en utilisant les essais des deux groupes,
en ajustant les paramètres des distributions pour mieux modéliser les valeurs d'hyperparamètres qui ont conduit à de bonnes performances.

3 - Échantillonnage des essais :
Une fois que les distributions de probabilité ont été mises à jour, 
TPE échantillonne de nouvelles valeurs d'hyperparamètres à partir de ces distributions. 
Les valeurs échantillonnées sont généralement celles qui maximisent l'espérance d'une fonction d'acquisition 
(par exemple, l'espérance de l'amélioration de l'objectif) 
ou qui minimisent une fonction de coût (par exemple, l'espérance de la perte de l'objectif).

4 - Évaluation des performances :
Les nouvelles valeurs d'hyperparamètres échantillonnées sont utilisées pour effectuer de nouveaux essais.
Les performances de ces essais sont évaluées à l'aide de la fonction d'objectif, 
et les résultats sont utilisés pour mettre à jour les distributions de probabilité et répéter le processus.
"""

'\nSAMPLER TPE :\nTree-structured Parzen Estimator\nTPESampler utilise une approche bayésienne pour estimer les distributions de probabilité des valeurs d\'hyperparamètres,\nen se basant sur les performances des essais précédents, \nafin de guider efficacement la recherche vers des régions prometteuses de l\'espace des hyperparamètres.\n\n1 - Initialisation des distributions :\nL\'algorithme TPE commence par définir des distributions de probabilité pour chaque hyperparamètre à optimiser.\nCes distributions peuvent être continues (comme des distributions gaussiennes) ou discrètes (comme des distributions uniformes).\nPour chaque hyperparamètre, deux distributions sont définies : \nune pour les valeurs considérées comme "bonnes" (positive), et une pour les valeurs considérées comme "mauvaises" (négative).\n\n2 - Évaluation des essais :\nÀ chaque étape de l\'optimisation, \nTPE utilise les essais précédents pour estimer les distributions de probabilité des valeurs d\'hyperparamètres.\nLes

In [24]:
"""
Démarrer un serveur mlflow local en ligne de commande :
mlflow server --host 127.0.0.1 --port 8080
"""

'\nDémarrer un serveur mlflow local en ligne de commande :\nmlflow server --host 127.0.0.1 --port 8080\n'

In [42]:
# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri(f"{LOCAL_HOST}:{LOCAL_PORT}")

In [59]:
train.shape

(307507, 794)

In [90]:
# On crèe une epérience
experiment_name = f"lightgbm_hyperparam_simple_{len(predictors)}_{train.shape[0]}"
print(experiment_name)

experiment_description = (
    "Recherche d'hyperparamètres pour le modèle LightGBM, impact du nombre de features"
    "Recherche Bayesienne - mono objectif - Hyperband"
)

experiment_tags = {
    "model": "lightgbm",
    "task": "hyperparam",
    "mlflow.note.content": experiment_description,
}

# On vérifie si l'expérience existe déjà
existing_experiment = mlflow.get_experiment_by_name(experiment_name)

if existing_experiment:
    print("Le nom de l'expérience existe déjà, on l'active")
    experiment_id = existing_experiment.experiment_id
    
else:
    print(f"Création de l'expérience {experiment_name}")
    experiment_id = mlflow.create_experiment(
        experiment_name,
        #artifact_location=os.path.join(MODEL_DIR, subdir).as_uri(),
        artifact_location=os.path.join(MODEL_DIR, subdir),
        tags=experiment_tags,
    )
# On active l'expérience
experiment_metadata = mlflow.set_experiment(experiment_id=experiment_id)
experiment_metadata

lightgbm_hyperparam_simple_20_307507
Le nom de l'expérience existe déjà, on l'active


<Experiment: artifact_location='file:///E:/Mes Documents/_Open Classroom/Code/p7/models/light_simple', creation_time=1715006893309, experiment_id='520338994331402696', last_update_time=1715006893309, lifecycle_stage='active', name='lightgbm_hyperparam_simple_20_307507', tags={'mlflow.note.content': "Recherche d'hyperparamètres pour le modèle LightGBM, "
                        'impact du nombre de featuresRecherche Bayesienne - '
                        'mono objectif - Hyperband',
 'model': 'lightgbm',
 'task': 'hyperparam'}>

In [61]:
print(experiment_metadata.name)
print(experiment_metadata.lifecycle_stage)
print(experiment_metadata.artifact_location)

lightgbm_hyperparam_simple_20_307507
active
file:///E:/Mes Documents/_Open Classroom/Code/p7/models/light_simple


In [91]:
with timer("Optimize hyperparameters"):
    # Utilise l'algorithme d'optimisation TPE (Tree-structured Parzen Estimator) comme méthode d'échantillonnage
    # Il s'agit de l'algo qui génère les valeurs des hyperparams lors de chaque essai d'optimisation
    # Ici il est utilisé en conjonction avec le pruning Hyperband
    # => Le sampler choisit les params à essayer, le pruner les arrête prématurément si non performants
    sampler = optuna.samplers.TPESampler()
    
    # Optuna a réalisé plusieurs études empiriques avec différents algorithmes de pruning.
    # Empiriquement, l'algorithme Hyperband a donné les meilleurs résultats
    # Voir : https://github.com/optuna/optuna/wiki/Benchmarks-with-Kurobako
    # reduction_factor contrôle combien de trials sont proposés dans chaque Halving Round
    pruner = optuna.pruners.HyperbandPruner(
        min_resource=10, max_resource=400, reduction_factor=3)
    
    # On crèe une run MLFlow
    run_name = f"lightgbm_single_weighted_recall_{n_features}_best"
    run_description = f'Single objective - métrique f1_macro - all data kernel simple - {n_features}_features'
    
    
    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True) as run:
        # description du run
        mlflow.set_tag("mlflow.note.content", run_description)
        
        study = optuna.create_study(
            direction='maximize', 
            sampler=sampler,
            pruner=pruner,
            study_name="lightgbm_single_obj_03",
            #storage=os.path.join(MODEL_DIR, subdir)
        )
    
        # gc appelle le garbage collector après chaque trial
        study.optimize(objective(optimize_boosting_type=True), n_trials=100, gc_after_trial=True, n_jobs=-1)
        
        best_params = study.best_trial.params
        best_score =study.best_trial.value
        mlflow.log_params(best_params)
        #mlflow.sklearn.log_model(study.best_trial.model, "Meilleur modèle")
        
        fig = optuna.visualization.plot_parallel_coordinate(study, params=["boosting_type", "num_leaves", "learning_rate", "n_estimators"])
        im_dir = os.path.join(MODEL_DIR, subdir)
        #fig.write_image(file=os.path.join(im_dir, "single_parallel_coordinates.png"), format="png", scale=6)
        fig.write_html(os.path.join(im_dir, "single_parallel_coordinates.html"))
        fig = optuna.visualization.plot_param_importances(study)
        #fig.write_image(file=os.path.join(im_dir, "single_hyperparam_importance.png"), format="png", scale=1)
        fig.write_html(os.path.join(im_dir, "single_hyperparam_importance.html"))
        
        mlflow.log_params(best_params)
        mlflow.log_metric("weight_recall", best_score)
        mlflow.log_artifact(os.path.join(im_dir, "single_parallel_coordinates.html"))
        mlflow.log_artifact(os.path.join(im_dir, "single_hyperparam_importance.html"))
        #mlflow.log_artifact(os.path.join(im_dir, "single_hyperparam_importance.png"))
    # Force mlflow à terminer le run même s'il y a une erreur dedans
    mlflow.end_run()

[I 2024-05-06 18:20:15,182] A new study created in memory with name: lightgbm_single_obj_03
[I 2024-05-06 18:21:59,476] Trial 4 finished with value: 0.9192701304309328 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.03822282990343516, 'lambda_l2': 2.4351434935849822e-05, 'num_leaves': 52, 'feature_fraction': 0.6004499562553188, 'bagging_fraction': 0.790998968213686, 'bagging_freq': 7, 'min_child_samples': 83, 'learning_rate': 0.00026268426118726665, 'max_bin': 416, 'n_estimators': 80}. Best is trial 4 with value: 0.9192701304309328.
[I 2024-05-06 18:22:48,301] Trial 32 finished with value: 0.9192701304309328 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.00023901282533408106, 'lambda_l2': 0.0002595401544043731, 'num_leaves': 3, 'feature_fraction': 0.5679375961850424, 'bagging_fraction': 0.7351040716389725, 'bagging_freq': 2, 'min_child_samples': 55, 'learning_rate': 0.03258683746723998, 'max_bin': 512, 'n_estimators': 340}. Best is trial 4 with value: 0.9192701304309

Optimize hyperparameters - duration (hh:mm:ss) : 0:25:10


In [82]:
mlflow.end_run()

In [47]:
print(study.best_trial)
best_params = study.best_trial.params
for k, v in best_params.items():
    print(f"{k} : {v}")

FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.04807719281410693], datetime_start=datetime.datetime(2024, 5, 6, 15, 32, 46, 653851), datetime_complete=datetime.datetime(2024, 5, 6, 15, 35, 57, 587988), params={'boosting_type': 'dart', 'lambda_l1': 0.0002094385256850892, 'lambda_l2': 1.4244398909216404e-06, 'num_leaves': 165, 'feature_fraction': 0.5791097943448836, 'bagging_fraction': 0.4524519297389386, 'bagging_freq': 3, 'min_child_samples': 47, 'learning_rate': 0.4925837985942177, 'max_bin': 256, 'n_estimators': 200}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatD

In [70]:
joblib.dump(study, os.path.join(MODEL_DIR, subdir, "opt_lightgbm_single_03.pkl"))

['models/light_simple/opt_lightgbm_single_03.pkl']

## Understanding Parameters

In [72]:
study = joblib.load(os.path.join(MODEL_DIR, subdir, "opt_lightgbm_single_03.pkl"))

In [73]:
# Nécessite l'installation de plotly et de Kaleido (redémarrer le kernel impérativement après plotly, kaleido version 0.1.0 et nformat version récente)
# pip install kaleido==0.1.0
# pip install --upgrade nbformat
# Ajouter le path de kaleido.cmd au PATH windows
# Tout redémarrer
fig = optuna.visualization.plot_parallel_coordinate(study, params=["boosting_type", "num_leaves", "learning_rate", "n_estimators"])
joblib.dump(fig, os.path.join(MODEL_DIR,'fig_plotly.pkl'))
print(type(fig))

<class 'plotly.graph_objs._figure.Figure'>


In [74]:
im_dir = os.path.join(MODEL_DIR, subdir)

In [75]:
fig.write_image(file=os.path.join(im_dir, "single_parallel_coordinates.png"), format="png", scale=6)

In [76]:
fig.show()

In [44]:
# Plus rapide en html
im_path = os.path.join(im_dir, "single_parallel_coordinates.html")
fig.write_html(im_path)
fig.show()

In [78]:
fig = optuna.visualization.plot_param_importances(study)
fig.write_image(file=os.path.join(im_dir, "single_hyperparam_importance.png"), format="png", scale=6)
fig.show()

## Multi-objective optimization

In [22]:
def moo_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.5, log=True),

    model = lgb.LGBMClassifier(
        force_row_wise=True,
        boosting_type='gbdt',
        n_estimators=200,
        lambda_l1=3.298803078077973e-07,
        lambda_l2=8.938532783741386e-07,
        num_leaves=6,
        feature_fraction=0.5133218336120866,
        bagging_fraction=0.9660809666082303,
        bagging_freq=7,
        min_child_samples=91,
        learning_rate=learning_rate,
        max_bin=320,
        verbose=-1,
    )
    scores = cross_val_score(model, X, y, scoring="f1_macro")
    return learning_rate[0], scores.mean()

In [23]:
study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(moo_objective, n_trials=100)

[I 2023-04-27 10:47:35,501] A new study created in memory with name: no-name-eb122531-f2ee-4cb5-92d1-8efba6a28eef
[I 2023-04-27 10:47:36,221] Trial 0 finished with values: [0.018276625572235056, 0.7229728382357858] and parameters: {'learning_rate': 0.018276625572235056}. 
[I 2023-04-27 10:47:36,922] Trial 1 finished with values: [0.012281350875017864, 0.713129498994182] and parameters: {'learning_rate': 0.012281350875017864}. 
[I 2023-04-27 10:47:37,601] Trial 2 finished with values: [0.3179931543219117, 0.7181352358216049] and parameters: {'learning_rate': 0.3179931543219117}. 
[I 2023-04-27 10:47:38,317] Trial 3 finished with values: [0.3317048028124761, 0.7182462105095208] and parameters: {'learning_rate': 0.3317048028124761}. 
[I 2023-04-27 10:47:39,042] Trial 4 finished with values: [0.010097499163932589, 0.689595604293249] and parameters: {'learning_rate': 0.010097499163932589}. 
[I 2023-04-27 10:47:39,716] Trial 5 finished with values: [0.0010284892903758126, 0.4235082247366586]

In [24]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["learning_rate", "f1"])
fig.write_image(file="figures/ch5_pareto.png", format="png", scale=6)
fig.show()